In [1]:
import os
import sys
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

/Users/nikhil/Documents/GitHub_portfolio/1-Enterprise-Grade RAG System/notebook
/Users/nikhil/Documents/GitHub_portfolio/1-Enterprise-Grade RAG System


In [3]:
%%capture
!pip install -r requirements.txt

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings,OllamaLLM
from langchain_huggingface import HuggingFaceEmbeddings


In [7]:
def read_data(floder_path):
    all_docs =[]
    for file_name in os.listdir(floder_path):
        file_path = os.path.join(floder_path,file_name)
        if file_path.endswith(".pdf"):
            docs = PyMuPDFLoader(file_path)
            documents = docs.load()
            all_docs.extend(documents)
        else:
            print("Unknown file type:", file_path)
    return all_docs

In [9]:
folder_path = "./data/"
documents=read_data(folder_path)

Unknown file type: ./data/.ipynb_checkpoints
Unknown file type: ./data/qa_dataset.json


In [10]:
len(documents)

2147

In [11]:
import re
current_section = "unknown"
for doc in documents:
    match = re.search(r"(\d{4})",doc.metadata.get("file_path"," "))
    if match:
        doc.metadata['year']=int(match.group(0))
        doc.metadata['company'] = "Coca-cola"
        doc.metadata["filing_type"] = "10-K"
    text = doc.page_content.replace("\n"," ").replace("  "," ")
    match = re.search(r"(ITEM\s+\d+[A-Z]?\.\s+[A-Z\s,&\-]+)", text)
    if match:
        current_section = match.group(0).strip()
        doc.metadata['section']=current_section
        continue
    doc.metadata['section']=current_section



In [15]:
documents[9].metadata

{'producer': 'Qt 4.8.7',
 'creator': 'wkhtmltopdf 0.12.5',
 'creationdate': '2020-02-24T16:11:05-05:00',
 'source': './data/2020-cocacola-10k-filing.pdf',
 'file_path': './data/2020-cocacola-10k-filing.pdf',
 'total_pages': 214,
 'format': 'PDF 1.4',
 'title': 'Document',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '',
 'trapped': '',
 'modDate': '',
 'creationDate': "D:20200224161105-05'00'",
 'page': 9,
 'year': 2020,
 'company': 'Coca-cola',
 'filing_type': '10-K',
 'section': 'ITEM 1.  BUSINESS I'}

In [17]:
splitter = RecursiveCharacterTextSplitter( chunk_size=1024,chunk_overlap=100)

In [19]:
chunks = splitter.split_documents(documents)

In [21]:
len(chunks)

9532

In [23]:
hf = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')

In [24]:
vector_store = Chroma.from_documents(documents=chunks,embedding=hf,persist_directory =".db")

In [25]:
def get_documents_by_year_from_query(query):
    # Extract 4-digit year from query
    match = re.search(r"\b(\d{4})\b", query)
    years = [2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025]
    
    if match in years:
        year = int(match.group(1))  # Extract the matched year as integer
        retriever = vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={'k': 5, 'filter': {'year': year}}
        )
        results = retriever.invoke(query)
        return results
    else:
        retriever = vector_store.as_retriever(
            search_type="similarity",
        )
        results = retriever.invoke(query)
        return results
            
        
        
        
        

In [27]:
import re

def get_documents_by_year_from_query(query):
    # List of valid years
    years = [2015, 2016, 2017, 2018, 2019, 2020, 
             2021, 2022, 2023, 2024, 2025]
    
    # Extract 4-digit year from query
    match = re.search(r"\b(\d{4})\b", query)
    
    if match:
        year = int(match.group(1))  # Extract year as integer
        
        if year in years:
            retriever = vector_store.as_retriever(
                search_type="similarity",
                search_kwargs={'k': 5, 'filter': {'year': year+1}}
            )
        elif year<2015:
            retriever = vector_store.as_retriever(search_type="similarity",
                                                  search_kwargs={'k': 5, 'filter': {'year': 2015}}
                                                 )
    else:
        retriever = vector_store.as_retriever(search_type="similarity")
    
    results = retriever.invoke(query)
    return results


In [51]:
query = "What was the average number of diluted shares outstanding in 2014?"
ans = get_documents_by_year_from_query(query)

In [55]:
len(ans)

5

In [57]:
for i in ans:
    print(i.page_content)
    print(" ")
    print(i.metadata)
    print("-"*100)

$
1.90
$
1.97
AVERAGE SHARES OUTSTANDING
4,387
4,434
4,504
Effect of dilutive securities
63
75
80
AVERAGE SHARES OUTSTANDING ASSUMING DILUTION
4,450
4,509
4,584
1 Calculated based on net income attributable to shareowners of The Coca-Cola
Company.
Refer to Notes to Consolidated Financial Statements.
73
 
{'author': '', 'company': 'Coca-cola', 'creationDate': "D:20190503150839-04'00'", 'creationdate': '2019-05-03T15:08:39-04:00', 'creator': 'wkhtmltopdf 0.12.5', 'file_path': './data/2015-cocacola-10k-filing.pdf', 'filing_type': '10-K', 'format': 'PDF 1.4', 'keywords': '', 'modDate': '', 'moddate': '', 'page': 74, 'producer': 'Qt 4.8.7', 'section': 'ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA  TABLE OF CONTENTS P', 'source': './data/2015-cocacola-10k-filing.pdf', 'subject': '', 'title': '2014.12.31 10-K', 'total_pages': 216, 'trapped': '', 'year': 2015}
----------------------------------------------------------------------------------------------------
0.39
$
0.60
$
0.55
$
0.39
$

In [59]:
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
prompt_template = """
You are a finance analyst. You are responsible for generating a **factually correct answer** for the given question using the provided context.

Instructions:
- Use only the information from the context to generate the answer.
- Do **not** make up any facts.
- If the answer is not present in the context, respond with "The answer is not available in the provided context."

Question: {Question}

Context:
{context}
"""

In [61]:
prompt = PromptTemplate(
    input_variables=["Question", "context"],
    template=prompt_template
)

In [63]:
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser

llm = OllamaLLM(model="llama3.2:latest")
parser =StrOutputParser()

In [77]:
chain = prompt|llm

In [67]:
def answer_from_rag(question, chain):
    docs = get_documents_by_year_from_query(question)
    context = "\n\n".join([doc.page_content for doc in docs])
    context_list = [doc.page_content for doc in docs]
    rag_answer = chain.invoke({"Question": question, "context": context})
    return rag_answer ,context_list

In [121]:
question = "What was the average number of diluted shares outstanding in 2014?"
a,b=answer_from_rag(question, chain)

In [123]:
print(a)

The answer is not available in the provided context.


In [53]:
with open("coca_cola_2015_qa.json", "r") as f:
    dataset = json.load(f)

In [55]:
from deepeval import evaluate
from deepeval.test_case import LLMTestCase

from deepeval.metrics import FaithfulnessMetric,AnswerRelevancyMetric,ContextualPrecisionMetric,ContextualRecallMetric,ContextualRelevancyMetric

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/anaconda3/lib/python3.12/site-packages/deepeval/__init__.py:54: UserWarning: You are using deepeval version 2.6.5, however version 3.4.8 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [57]:
test_cases = []
for item in dataset:
    rag_answer,retrieved_docs = answer_from_rag(item["question"],chain=chain)
    
    test_cases.append(
        LLMTestCase(
            input=item["question"],                       # the query
            actual_output=rag_answer,                  # RAG's generated answer
            expected_output=item["answer"],  # gold answer
            retrieval_context=retrieved_docs,     # what your retriever returned
        )
    )

In [58]:
!deepeval set-ollama gemma3:latest

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/anaconda3/lib/python3.12/site-packages/deepeval/__init__.py:54: UserWarning: You are using deepeval version 2.6.5, however version 3.4.8 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(
🙌 Congratulations! You're now using a local Ollama model for all evals that 
require an LLM.


In [59]:
metrics = [
    AnswerRelevancyMetric(),
    FaithfulnessMetric(),
    ContextualPrecisionMetric(),
    ContextualRecallMetric(),
    ContextualRelevancyMetric()
    
]

In [60]:
evaluate(test_cases=test_cases, metrics=metrics)

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gemma3:latest (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gemma3:latest (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using gemma3:latest (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using gemma3:latest (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gemma3:latest (Ollama), strict=False, 
async_mode=True)...

Evaluating 10 test case(s) in parallel: | |  0% (0/10) [Time Taken: 00:00, ?test

KeyboardInterrupt: 

In [ ]:
import json
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams
correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the expected output.",
    # NOTE: you can only provide either criteria or evaluation_steps, and not both
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'expected output'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
)

In [227]:
evaluate(test_cases=test_cases, metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness (GEval) Metric! (using gemma3:latest (Ollama), strict=False, 
async_mode=True)...

Evaluating 4 test case(s) in parallel: |█|100% (4/4) [Time Taken: 00:07,  2.00s/



Metrics Summary

  - ✅ Correctness (GEval) (score: 0.7, threshold: 0.5, strict: False, evaluation model: gemma3:latest (Ollama), reason: The actual output lists several competitors, including Nestlé, DPSG, Groupe Danone, Mondelēz, Kraft, Suntory, and Unilever, aligning with the expected output. However, PepsiCo is missing, a key competitor listed in the expected output. The actual output also includes a general statement about potential impacts, while the expected output is slightly more specific regarding investment in marketing and innovation., error: None)

For test case:

  - input: Which companies are considered major competitors to The Coca-Cola Company in the nonalcoholic beverage segment, and how might this competition impact their market share in 2014?
  - actual output: Based on the provided context, the major competitors to The Coca-Cola Company in the nonalcoholic beverage segment include:

1. Nestlé
2. DPSG (The Dr Pepper Snapple Group)
3. Groupe Danone
4. Mondelēz
5. Kr

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_1', success=True, metrics_data=[MetricData(name='Correctness (GEval)', threshold=0.5, success=True, score=0.7, reason='The actual output lists several competitors, including Nestlé, DPSG, Groupe Danone, Mondelēz, Kraft, Suntory, and Unilever, aligning with the expected output. However, PepsiCo is missing, a key competitor listed in the expected output. The actual output also includes a general statement about potential impacts, while the expected output is slightly more specific regarding investment in marketing and innovation.', strict_mode=False, evaluation_model='gemma3:latest (Ollama)', error=None, evaluation_cost=0.0, verbose_logs='Criteria:\nDetermine whether the actual output is factually correct based on the expected output. \n \nEvaluation Steps:\n[\n    "Check whether the facts in \'actual output\' contradicts any facts in \'expected output\'",\n    "You should also heavily penalize omission of detail",\n    "Vague lan